In [9]:
print("hi sheriff")

hi sheriff


In [10]:
%pwd

'e:\\GENAI projects\\END_TO_END_MEDICAL_CHATBOT_USING_OPENAI\\research'

In [11]:
import os
os.chdir("../")

In [12]:
%pwd

'e:\\GENAI projects\\END_TO_END_MEDICAL_CHATBOT_USING_OPENAI'

In [13]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [15]:
extracted_data=load_pdf_file(data='Data/')

In [16]:
#split the data into Chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [17]:
text_chunks = text_split(extracted_data)
print("length of text Chunks",len(text_chunks))

length of text Chunks 10289


In [18]:
#Embedding the text
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
#Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [20]:
embeddings = download_hugging_face_embeddings()

C:\Users\mdshe\AppData\Local\Temp\ipykernel_39276\2145507391.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
c:\Users\mdshe\anaconda3\envs\llmaap\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
query_result = embeddings.embed_query("hello world")
print("length",len(query_result))

length 384


In [22]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [38]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [39]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [40]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain.vectorstores import Pinecone


docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [41]:
# Load Existing index 

from langchain_pinecone import Pinecone
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [42]:
docsearch

In [43]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [44]:
retrieved_docs = retriever.invoke("What is Acne?")

In [45]:
retrieved_docs

[Document(metadata={'page': 298.0, 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}, page_content='Resources\nBOOKS\nSwerlick, Robert A., and Thomas J. Lawley. “Eczema, Psoriasis,\nCutaneous Infections, Acne, and Other Common Skin Dis-\norders.” In Harrison’s Principles of Internal Medicine,e d .\nAnthony S. Fauci, et al. New York: McGraw-Hill, 1997.\nWolf Jr., John E. “Contact Dermatitis.” In Conn’s Current\nTherapy, 1996, ed. Robert E. Rakel. Philadelphia: W. B.\nSaunders Co., 1996.\nPERIODICALS\nBeltrani, Vincent S., and Vincent P. Beltrani. “Contact Dermati-'),
 Document(metadata={'page': 425.0, 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}, page_content='Corticosteriod—A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied t

In [46]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [51]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [52]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [53]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])



Acromegaly and gigantism are both conditions caused by overproduction of growth hormone from the pituitary gland, which can lead to excessive growth of bones and tissues. Acromegaly typically occurs in adults, while gigantism occurs in children and adolescents. Both conditions can cause a variety of symptoms, including enlarged hands and feet, facial changes, and health complications such as high blood pressure and diabetes.


In [55]:
response = rag_chain.invoke({"input": "What is sheriff?"})
print(response["answer"])


A sheriff is a law enforcement officer who is responsible for maintaining peace and order in a specific county or jurisdiction. They often have the authority to make arrests, serve warrants, and enforce court orders. In some areas, the sheriff may also oversee the operation of the county jail.
